.

#### 1. 개요

- S3 Sync 기능을 이용한 마이그레이션 테스트


#### 2. 활용가능한 시나리오 예시

1) 목적

AS-IS 계정에 있는 S3의 모든 버킷과 데이터를 같은 리전의 TO-BE계정으로 데이터 마이그레이션

(S3 to S3 Migration)

2) 전제조건


- S3 총 데이터 규모가 수십TB 단위


- AS-IS 계정은 현재 서비스 중이기 때문에 AS-IS 계정의 네트워크나 config 설정을 건드는 것을 최소화해야함


- 1차 베이스라인 데이터 백업 후 디데이까지 일배치로 AS-IS 계정의 s3 데이터와 TO-BE 계정의 s3 데이터 간 sync를 맞추는 작업 필요


#### 3. S3 Sync 기능 테스트

STEP 1) TO-BE 계정의 Destination 버킷에 대하여 AS-IS 계정이 접근할 수 있도록 버킷권한 설정

- TO-BE 계정의 Destination 버킷에서 아래와 같이 버킷권한 설정

![22](https://user-images.githubusercontent.com/41605276/76311599-abbce300-6314-11ea-8030-31f2abf3b99c.png)

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "DelegateS3Access",
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::111111111111:user/pms-s3synctest-user"
            },
            "Action": [
                "s3:ListBucket",
                "s3:GetObject",
                "s3:PutObject",
                "s3:DeleteObject",
                "s3:GetObjectAcl"
            ],
            "Resource": [
                "arn:aws:s3:::minman2",
                "arn:aws:s3:::minman2/*"
            ]
        }
    ]
}

STEP 2) AS-IS 계정에서 아래와 같이 사용자계정 inline 정책을 만들어서 부여

![asis](https://user-images.githubusercontent.com/41605276/77998683-6ec09b00-736c-11ea-8a5a-440b34ea4bb6.png)

** STEP 1 ~ 2) 참고자료 : https://aws.amazon.com/ko/premiumsupport/knowledge-center/copy-s3-objects-account/

STEP 3) TO-BE 계정에서 EC2(amazon linux2) 생성 후 task 부여환경 구성

step 3-1) ec2 생성 및 ssh 접속

step 3-2) 파이썬 3.7 및 nmon 설치

In [ ]:
sudo yum update -y
sudo yum install python37 python37-pip -y
sudo amazon-linux-extras install epel -y
sudo yum update -y
sudo yum install nmon -y
sudo pip3 install boto3

step 3-3) os 파이썬 기본버전 변경 (2.7 -> 3.7)

In [ ]:
sudo alternatives --install /usr/bin/python python /usr/bin/python2.7 1
sudo alternatives --install /usr/bin/python python /usr/bin/python3.7 2

step 3-4) 주피터 설치

In [ ]:
sudo pip3 install jupyter

step 3-5) 주피터 설정파일 생성

In [ ]:
jupyter notebook --generate-config

step 3-6) 설정파일 내 설정 변경

In [ ]:
sudo vi /home/ec2-user/.jupyter/jupyter_notebook_config.py

step 3-6-1) 외부 접속 허용

In [ ]:
##  Takes precedence over allow_origin_pat.
c.NotebookApp.allow_origin = '*'

step 3-6-2) 작업 경로 설정

In [ ]:
## The directory to use for notebooks and kernels.
c.NotebookApp.notebook_dir = '/home/ec2-user'

step 3-6-3) 시작시 브라우저 실행 안함설정

In [ ]:
## Whether to open in a browser after starting.
c.NotebookApp.open_browser = False

step 3-6-4) 접속 암호 설정

암호 설정을 위해 아래와 같이 암호 생성

리눅스 콘솔에서 python 실행

아래와 같이 입력

In [ ]:
Python 3.6.1 |Continuum Analytics, Inc.| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Type "help", "copyright", "credits" or "license" for more information.
>>> from notebook.auth import passwd
>>> passwd()
Enter password: 1234
Verify password: 1234
'sha1:**********************'

step 3-6-5) 다시 설정파일로 들어가서 비밀번호 설정부분 변경

In [ ]:
#  The string should be of the form type:salt:hashed-password.
c.NotebookApp.password = 'sha1:***********************'

step 3-6-6) 접속포트 설정

- 포트설정과 동시에 ec2 보안그룹에서도 8800 포트를 열어준다.

In [ ]:
## The port the notebook server will listen on.
c.NotebookApp.port = 8800

step 3-7) 리눅스 콘솔에서 aws configure 설정

step 3-8) 시스템 타임존 변경

tzselect 명령어를 사용하여 타임존을 서울 시간으로 설정

(이 방법은 로컬에만 적용된다는 것을 주의한다)

In [ ]:
echo "TZ='Asia/Seoul'; export TZ" >> .profile
. .profile
date

step 3-9) 아래와 같이 리눅스 콘솔에서 주피터 실행

In [ ]:
jupyter notebook --ip=0.0.0.0 --port=8800

아래와 같이 브라우저에서 IP주소:8800으로 접속

![JYPY](https://user-images.githubusercontent.com/41605276/76313004-78c81e80-6317-11ea-9628-fe8a84e9748e.png)

STEP 4) jupyter에서 노트북 생성 후 아래와 같은 코드를 작성하여 코드 실행

In [23]:
import subprocess
from datetime import datetime
import time
import boto3
import math

start = time.time()

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')+'\n'
start_timestamp = timestamp[0:4]+timestamp[5:7]+timestamp[8:10]

source_bucket = "k"
source_prefix = "m"
source_s3_location = source_bucket+'/'+source_prefix

destination_bucket = 'a'
destination_prefix = 'b/d'
destination_s3_location = destination_bucket +'/'+ destination_prefix
task_name = "task 3"

sync_command = f"aws s3 sync s3://{source_s3_location}/ s3://{destination_s3_location}/ --delete --acl bucket-owner-full-control"

file_name = start_timestamp+" "+task_name

logfile = open("{}.txt".format(file_name), 'wb')
start_time = "start time : {}".format(timestamp).encode()
logfile.write(start_time)

try :
    log_record = subprocess.check_output(sync_command, stderr=subprocess.STDOUT, shell=True)
    print("job success")
    logfile.write(log_record)
    runtime = "Runtime: {} sec \n".format(time.time()-start).encode()
    logfile.write(runtime)
    print(runtime)
    
    # check data integrity
    client = boto3.client('s3')
    paginator = client.get_paginator('list_objects_v2')
    
    operation_parameters = {'Bucket': '{}'.format(source_bucket),'Prefix': '{}/'.format(source_prefix)}
    response_iterator = paginator.paginate(**operation_parameters)
    source_object_list = []
    source_folder_list = []

    for page in response_iterator:
        for content in page['Contents']:
            if (content['Key']+','+str(content['Size']))[-3:] == '/,0':
                source_folder_list.append(content['Key']+','+str(content['Size']))
            else : 
                source_object_list.append(content['Key'].split('/').pop()+','+str(content['Size']))
  

    operation_parameters = {'Bucket': '{}'.format(destination_bucket),'Prefix': '{}/'.format(destination_prefix)}
    response_iterator = paginator.paginate(**operation_parameters)
    destination_object_list = []
    destination_folder_list = []

    for page in response_iterator:
        for content in page['Contents']:
            if (content['Key']+','+str(content['Size']))[-3:] == '/,0':
                destination_folder_list.append(content['Key']+','+str(content['Size']))
            else : 
                destination_object_list.append(content['Key'].split('/').pop()+','+str(content['Size']))

    if source_object_list == destination_object_list :
# 실시간 데이터가 유입되는 버킷은 아래와 같이 정합성 테스트
#     if source_object_list[0:math.floor(len(source_object_list)*0.95)] == destination_object_list[0:math.floor(len(source_object_list)*0.95)] : 

        logfile.write("data integrity check success \n".encode())
        logfile.write("{} to {} complete \n".format(source_s3_location,destination_s3_location).encode())
        
        source_folder_list_str = "source_folder_list:" + str(source_folder_list)+'\n'
        logfile.write(source_folder_list_str.encode())
        source_object_list_str = "source_object_list:" + str(source_object_list)+'\n'
        logfile.write(source_object_list_str.encode())
        
        destination_folder_list_str = "destination_folder_list:"+str(destination_folder_list)+'\n'
        logfile.write(destination_folder_list_str.encode())
        destination_object_list_str = "destination_object_list:"+str(destination_object_list)+'\n'
        logfile.write(destination_object_list_str.encode())
        
        source_object_list_str = "number of source_object_list:" + str(len(source_object_list))+'\n'
        logfile.write(source_object_list_str.encode())
        destination_object_list_str = "number of destination_object_list:" + str(len(destination_object_list))+'\n'
        logfile.write(destination_object_list_str.encode())
        
        print("data integrity check success \n")
    else:
        logfile.write("data integrity check fail \n".encode())
        
        source_folder_list_str = "source_folder_list:" + str(source_folder_list)+'\n'
        logfile.write(source_folder_list_str.encode())
        source_object_list_str = "source_object_list:" + str(source_object_list)+'\n'
        logfile.write(source_object_list_str.encode())
        
        destination_folder_list_str = "destination_folder_list:"+str(destination_folder_list)+'\n'
        logfile.write(destination_folder_list_str.encode())
        destination_object_list_str = "destination_object_list:"+str(destination_object_list)+'\n'
        logfile.write(destination_object_list_str.encode())
        
        source_object_list_str = "number of source_object_list:" + str(len(source_object_list))+'\n'
        logfile.write(source_object_list_str.encode())
        destination_object_list_str = "number of destination_object_list:" + str(len(destination_object_list))+'\n'
        logfile.write(destination_object_list_str.encode())
        
        print("data integrity check fail")
    
except subprocess.CalledProcessError as e:
    print("job fail")
    logfile.write(e.output)
    runtime = "Runtime: {} sec".format(time.time()-start).encode()
    logfile.write(runtime)
    print(runtime)
    
logfile.close()

job success
b'Runtime: 1.083536148071289 sec \n'
data integrity check success 



로그파일 저장된 내용 예시

![aa](https://user-images.githubusercontent.com/41605276/76387930-410ab680-63ab-11ea-9c6c-6cea8389d1e7.png)